In [57]:
import argparse
import dask
import json
import netCDF4 as nc4
import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint
import time
import warnings
import xarray as xr
from dask import delayed
import datetime
import random
from pandas import read_csv

warnings.filterwarnings('ignore')

In [76]:
ecco_V4r4_metadata_dir = Path('/home/ifenty/git_repos_others/ECCO-GROUP/ECCO-ACCESS/metadata/ECCOv4r4_metadata_json')
dataset_metadata_fname = dataset_metadata_dir / 'PODAAC_datasets-revised_20210226.5.csv'

In [78]:
# load PODAAC fields
podaac_dataset_table = read_csv(dataset_info)

In [79]:
ds_shortname = list(podaac_dataset_table['DATASET.SHORT_NAME'])

In [80]:
ds_summary = list(podaac_dataset_table['DATASET.DESCRIPTION'])

In [81]:
ds_title = list(podaac_dataset_table['DATASET.LONG_NAME'])

In [82]:
ds_fname = list(podaac_dataset_table['DATASET.FILENAME'])

In [109]:
summary_second_part = 'Estimating the Circulation and Climate of the Ocean (ECCO) state estimates are dynamically and kinematically-consistent reconstructions of the three-dimensional, time-evolving ocean, sea-ice, and surface atmospheric states. ECCO V4r4 is a free-running solution of a global, nominally 1-degree configuration of the MIT general circulation model (MITgcm) that has been fit to observations in a least-squares sense. Observational data constraints used in V4r4 include sea surface height (SSH) from satellite altimeters [ERS-1/2, TOPEX/Poseidon, GFO, ENVISAT, Jason-1,2,3, CryoSat-2, and SARAL/AltiKa]; sea surface temperature (SST) from satellite radiometers [AVHRR], sea surface salinity (SSS) from the Aquarius satellite radiometer/scatterometer, ocean bottom pressure (OBP) from the GRACE satellite gravimeter; sea-ice concentration from satellite radiometers [SSM/I and SSMIS], and in-situ ocean temperature and salinity measured with conductivity-temperature-depth (CTD) sensors and expendable bathythermographs (XBTs) from several programs [e.g., WOCE, GO-SHIP, Argo, and others] and platforms [e.g., research vessels, gliders, moorings, ice-tethered profilers, and instrumented pinnipeds]. V4r4 covers the period 1992-01-01T12:00:00 to 2018-01-01T00:00:00.'
print(summary_second_part)

Estimating the Circulation and Climate of the Ocean (ECCO) state estimates are dynamically and kinematically-consistent reconstructions of the three-dimensional, time-evolving ocean, sea-ice, and surface atmospheric states. ECCO V4r4 is a free-running solution of a global, nominally 1-degree configuration of the MIT general circulation model (MITgcm) that has been fit to observations in a least-squares sense. Observational data constraints used in V4r4 include sea surface height (SSH) from satellite altimeters [ERS-1/2, TOPEX/Poseidon, GFO, ENVISAT, Jason-1,2,3, CryoSat-2, and SARAL/AltiKa]; sea surface temperature (SST) from satellite radiometers [AVHRR], sea surface salinity (SSS) from the Aquarius satellite radiometer/scatterometer, ocean bottom pressure (OBP) from the GRACE satellite gravimeter; sea-ice concentration from satellite radiometers [SSM/I and SSMIS], and in-situ ocean temperature and salinity measured with conductivity-temperature-depth (CTD) sensors and expendable bath

In [113]:
ds_summary_fix = dict()
for sni, sn in enumerate(ds_shortname):
    print(sni, sn, dataset_title[sni][0:50])
    ds_summary_fix[sn] = dict()
    ds_summary_fix[sn]['title'] = ds_title[sni]
    ds_summary_fix[sn]['summary_first_part']= ds_summary[sni].split('Estimating the Circulation and Climate of the Ocean')[0]
    ds_summary_fix[sn]['summary']= ds_summary_fix[sn]['summary_first_part'] + summary_second_part
    ds_summary_fix[sn]['filename'] = ds_fname[sni]

0 ECCO_L4_ATM_STATE_05DEG_DAILY_V4R4 ECCO Atmosphere Surface Temperature, Humidity, Win
1 ECCO_L4_ATM_STATE_05DEG_MONTHLY_V4R4 ECCO Atmosphere Surface Temperature, Humidity, Win
2 ECCO_L4_ATM_STATE_LLC0090GRID_DAILY_V4R4 ECCO Atmosphere Surface Temperature, Humidity, Win
3 ECCO_L4_ATM_STATE_LLC0090GRID_MONTHLY_V4R4 ECCO Atmosphere Surface Temperature, Humidity, Win
4 ECCO_L4_BOLUS_05DEG_DAILY_V4R4 ECCO Gent-McWilliams Ocean Bolus Velocity - Daily 
5 ECCO_L4_BOLUS_05DEG_MONTHLY_V4R4 ECCO Gent-McWilliams Ocean Bolus Velocity - Monthl
6 ECCO_L4_BOLUS_LLC0090GRID_DAILY_V4R4 ECCO Gent-McWilliams Ocean Bolus Velocity - Daily 
7 ECCO_L4_BOLUS_LLC0090GRID_MONTHLY_V4R4 ECCO Gent-McWilliams Ocean Bolus Velocity - Monthl
8 ECCO_L4_DENS_STRAT_PRESS_05DEG_DAILY_V4R4 ECCO Ocean Density, Stratification, and Hydrostati
9 ECCO_L4_DENS_STRAT_PRESS_05DEG_MONTHLY_V4R4 ECCO Ocean Density, Stratification, and Hydrostati
10 ECCO_L4_DENS_STRAT_PRESS_LLC0090GRID_DAILY_V4R4 ECCO Ocean Density, Stratification, a

In [114]:
ds_summary_fix[sn]['summary_first_part']

'This dataset provides 3D time-invariant coefficients for the Gent-McWilliams and Redi parameterizations and background vertical diffusivity interpolated to a regular 0.5-degree grid from the ECCO Version 4 Release 4 (V4r4) ocean and sea-ice state estimate. '

In [115]:
ds_summary_fix[sn]['summary']

'This dataset provides 3D time-invariant coefficients for the Gent-McWilliams and Redi parameterizations and background vertical diffusivity interpolated to a regular 0.5-degree grid from the ECCO Version 4 Release 4 (V4r4) ocean and sea-ice state estimate. Estimating the Circulation and Climate of the Ocean (ECCO) state estimates are dynamically and kinematically-consistent reconstructions of the three-dimensional, time-evolving ocean, sea-ice, and surface atmospheric states. ECCO V4r4 is a free-running solution of a global, nominally 1-degree configuration of the MIT general circulation model (MITgcm) that has been fit to observations in a least-squares sense. Observational data constraints used in V4r4 include sea surface height (SSH) from satellite altimeters [ERS-1/2, TOPEX/Poseidon, GFO, ENVISAT, Jason-1,2,3, CryoSat-2, and SARAL/AltiKa]; sea surface temperature (SST) from satellite radiometers [AVHRR], sea surface salinity (SSS) from the Aquarius satellite radiometer/scatteromet

In [123]:
for sni, sn in enumerate(ds_summary_fix.keys()):
    print(f'\n{sni} : {sn}')
    print(ds_summary_fix[sn]['title'])
    print('--------------------------------------------------')
    print(' ' ,ds_summary_fix[sn]['summary_first_part'], '\n')


0 : ECCO_L4_ATM_STATE_05DEG_DAILY_V4R4
ECCO Atmosphere Surface Temperature, Humidity, Wind, and Pressure - Daily Mean 0.5 Degree (Version 4 Release 4)
--------------------------------------------------
  This dataset provides daily-averaged atmosphere surface temperature, humidity, wind, and pressure interpolated to a regular 0.5-degree grid from the ECCO Version 4 Release 4 (V4r4) ocean and sea-ice state estimate.  


1 : ECCO_L4_ATM_STATE_05DEG_MONTHLY_V4R4
ECCO Atmosphere Surface Temperature, Humidity, Wind, and Pressure - Monthly Mean 0.5 Degree (Version 4 Release 4)
--------------------------------------------------
  This dataset provides monthly-averaged atmosphere surface temperature, humidity, wind, and pressure interpolated to a regular 0.5-degree grid from the ECCO Version 4 Release 4 (V4r4) ocean and sea-ice state estimate.  


2 : ECCO_L4_ATM_STATE_LLC0090GRID_DAILY_V4R4
ECCO Atmosphere Surface Temperature, Humidity, Wind, and Pressure - Daily Mean llc90 Grid (Version 4 R

In [118]:
ds_summary_fix

{'ECCO_L4_ATM_STATE_05DEG_DAILY_V4R4': {'title': 'ECCO Atmosphere Surface Temperature, Humidity, Wind, and Pressure - Daily Mean 0.5 Degree (Version 4 Release 4)',
  'summary_first_part': 'This dataset provides daily-averaged atmosphere surface temperature, humidity, wind, and pressure interpolated to a regular 0.5-degree grid from the ECCO Version 4 Release 4 (V4r4) ocean and sea-ice state estimate. ',
  'summary': 'This dataset provides daily-averaged atmosphere surface temperature, humidity, wind, and pressure interpolated to a regular 0.5-degree grid from the ECCO Version 4 Release 4 (V4r4) ocean and sea-ice state estimate. Estimating the Circulation and Climate of the Ocean (ECCO) state estimates are dynamically and kinematically-consistent reconstructions of the three-dimensional, time-evolving ocean, sea-ice, and surface atmospheric states. ECCO V4r4 is a free-running solution of a global, nominally 1-degree configuration of the MIT general circulation model (MITgcm) that has be

In [119]:
output_dir = Path('/home/ifenty/git_repos_others/ECCO-GROUP/ECCO-ACCESS/metadata/ECCOv4r4_metadata_json')

In [127]:
with open(output_dir / 'ECCOv4r4_dataset_summary.json', 'w') as outfile:
    outfile.write(json.dumps(ds_summary_fix, indent=4))